# Mistral V2 Client Example

This notebook demonstrates how to use the Mistral V2 client (ModelClientV2 architecture) with AG2 agents.

## Key Features

- **Rich UnifiedResponse**: Returns `UnifiedResponse` with typed content blocks
- **Direct Property Access**: Use `response.text`, `response.get_content_by_type()` instead of parsing nested structures
- **Tool Call Support**: Preserves tool calls as `ToolCallContent` blocks
- **Backward Compatible**: Works seamlessly with existing AG2 agents

## Setup

Set your `MISTRAL_API_KEY` environment variable or provide it in the config.

In [ ]:
import os

# Configure Mistral V2 client
llm_config_mistral_v2 = {
    "config_list": [
        {
            "api_type": "mistral_v2",  # <-- Key: use V2 client architecture
            "model": "mistral-small-latest",
            "api_key": os.getenv("MISTRAL_API_KEY"),
        }
    ],
    "temperature": 0.7,
}

# For comparison: V1 Mistral client
llm_config_mistral_v1 = {
    "config_list": [
        {
            "api_type": "mistral",  # <-- V1 client architecture
            "model": "mistral-small-latest",
            "api_key": os.getenv("MISTRAL_API_KEY"),
        }
    ],
    "temperature": 0.7,
}

## Direct Client Usage

Let's use the Mistral V2 client directly to see the rich response format.

In [ ]:
from autogen.llm_clients import MistralAIClientV2
from autogen.llm_clients.models import UnifiedResponse

# Create client
client = MistralAIClientV2(api_key=os.getenv("MISTRAL_API_KEY"))

# Make a request
response = client.create({
    "model": "mistral-small-latest",
    "messages": [{"role": "user", "content": "Explain quantum computing in simple terms."}],
})

# Verify it's a UnifiedResponse
print(f"Response type: {type(response)}")
print(f"Is UnifiedResponse: {isinstance(response, UnifiedResponse)}")
print(f"\nProvider: {response.provider}")
print(f"Model: {response.model}")
print(f"\nText content: {response.text}")
print(f"\nUsage: {response.usage}")
print(f"Cost: ${response.cost:.6f}")

## Accessing Rich Content

The V2 client preserves all content in typed blocks. Let's explore the message structure.

In [ ]:
# Access message content
if response.messages:
    message = response.messages[0]
    print(f"Role: {message.role}")
    print(f"\nContent blocks ({len(message.content)}):")

    for i, block in enumerate(message.content):
        print(f"\n  Block {i + 1}:")
        print(f"    Type: {block.type}")
        print(f"    Class: {type(block).__name__}")

        if hasattr(block, "text"):
            print(f"    Text: {block.text[:100]}...")

        if hasattr(block, "name"):
            print(f"    Tool name: {block.name}")
            print(f"    Tool arguments: {block.arguments}")

## Tool Calls Example

Let's see how tool calls are preserved in the V2 format.

In [ ]:
# Example with tool calls
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {"location": {"type": "string", "description": "City name"}},
                "required": ["location"],
            },
        },
    }
]

response_with_tools = client.create({
    "model": "mistral-small-latest",
    "messages": [{"role": "user", "content": "What's the weather in Paris?"}],
    "tools": tools,
})

# Access tool calls
if response_with_tools.messages:
    message = response_with_tools.messages[0]
    tool_calls = message.get_tool_calls()

    print(f"Text content: {message.get_text()}")
    print(f"\nTool calls found: {len(tool_calls)}")

    for tool_call in tool_calls:
        print("\n  Tool Call:")
        print(f"    ID: {tool_call.id}")
        print(f"    Name: {tool_call.name}")
        print(f"    Arguments: {tool_call.arguments}")

        # Parse arguments
        import json

        args = json.loads(tool_call.arguments)
        print(f"    Parsed args: {args}")

## Using with ConversableAgent

The Mistral V2 client works seamlessly with AG2 agents. The agent automatically handles the rich response format.

In [ ]:
from autogen import ConversableAgent

# Create an agent using Mistral V2 client
agent = ConversableAgent(
    name="mistral_assistant",
    llm_config=llm_config_mistral_v2,
    system_message="You are a helpful assistant powered by Mistral AI.",
    human_input_mode="NEVER",
)

# Chat with the agent
reply = agent.generate_reply(messages=[{"role": "user", "content": "Write a haiku about artificial intelligence."}])

print("Agent Reply:")
print(reply)

## Backward Compatibility: V1 Compatible Format

The V2 client can also return responses in the legacy ChatCompletion format for compatibility.

In [ ]:
# Get V1 compatible response
v1_response = client.create_v1_compatible({
    "model": "mistral-small-latest",
    "messages": [{"role": "user", "content": "Hello!"}],
})

print("V1 Compatible Response:")
print(f"Type: {type(v1_response)}")
print("\nStructure:")
print(f"  ID: {v1_response['id']}")
print(f"  Model: {v1_response['model']}")
print(f"  Object: {v1_response['object']}")
print(f"  Choices: {len(v1_response['choices'])}")
if v1_response["choices"]:
    print(f"  Content: {v1_response['choices'][0]['message']['content']}")

## Message Retrieval

The `message_retrieval()` method works with UnifiedResponse and returns text or dicts based on content complexity.

In [ ]:
# Simple text response
response = client.create({
    "model": "mistral-small-latest",
    "messages": [{"role": "user", "content": "Say hello"}],
})

messages = client.message_retrieval(response)
print(f"Retrieved messages: {messages}")
print(f"Type: {type(messages[0])}")

# Response with tool calls
response_with_tools = client.create({
    "model": "mistral-small-latest",
    "messages": [{"role": "user", "content": "Get weather in NYC"}],
    "tools": tools,
})

messages_with_tools = client.message_retrieval(response_with_tools)
print(f"\nRetrieved messages with tools: {messages_with_tools}")
print(f"Type: {type(messages_with_tools[0])}")
if isinstance(messages_with_tools[0], dict):
    print(f"  Has tool_calls: {'tool_calls' in messages_with_tools[0]}")

## Cost and Usage Tracking

The V2 client provides easy access to usage statistics and cost calculation.

In [ ]:
response = client.create({
    "model": "mistral-small-latest",
    "messages": [{"role": "user", "content": "Write a short story about a robot."}],
})

# Direct access to cost
print(f"Cost: ${response.cost:.6f}")

# Get detailed usage
usage = client.get_usage(response)
print("\nUsage Details:")
for key, value in usage.items():
    print(f"  {key}: {value}")

# Calculate cost separately
calculated_cost = client.cost(response)
print(f"\nCalculated cost: ${calculated_cost:.6f}")

## Group Chat Example

Mistral V2 client works seamlessly in group chats with other agents.

In [ ]:
from autogen import GroupChat, GroupChatManager

# Create agents with Mistral V2
writer = ConversableAgent(
    name="writer",
    llm_config=llm_config_mistral_v2,
    system_message="You are a creative writer. Write engaging content.",
    description="Writes creative content",
)

editor = ConversableAgent(
    name="editor",
    llm_config=llm_config_mistral_v2,
    system_message="You are an editor. Review and improve writing. Say DONE! when satisfied.",
    description="Reviews and edits content",
)

# Setup group chat
groupchat = GroupChat(
    agents=[writer, editor],
    speaker_selection_method="auto",
    messages=[],
)

manager = GroupChatManager(
    name="manager",
    groupchat=groupchat,
    llm_config=llm_config_mistral_v2,
    is_termination_msg=lambda x: "DONE!" in (x.get("content", "") or "").upper(),
)

# Start conversation
result = writer.initiate_chat(
    recipient=manager,
    message="Write a blog post about the benefits of renewable energy.",
)

print("\nChat History:")
for msg in result.chat_history:
    print(f"\n{msg['role']}: {msg.get('content', '')[:100]}...")

## Comparison: V1 vs V2 Client

Let's compare the response formats between V1 and V2 clients.

In [ ]:
from autogen.oai import OpenAIWrapper

# V2 Client
wrapper_v2 = OpenAIWrapper(config_list=llm_config_mistral_v2["config_list"])
response_v2 = wrapper_v2.create(messages=[{"role": "user", "content": "Hello!"}], cache_seed=None)

print("V2 Response:")
print(f"  Type: {type(response_v2)}")
print(f"  Has .text property: {hasattr(response_v2, 'text')}")
if hasattr(response_v2, "text"):
    print(f"  Text: {response_v2.text}")

# V1 Client
wrapper_v1 = OpenAIWrapper(config_list=llm_config_mistral_v1["config_list"])
response_v1 = wrapper_v1.create(messages=[{"role": "user", "content": "Hello!"}], cache_seed=None)

print("\nV1 Response:")
print(f"  Type: {type(response_v1)}")
print(f"  Has .text property: {hasattr(response_v1, 'text')}")

# Both work with OpenAIWrapper.extract_text_or_completion_object()
text_v2 = wrapper_v2.extract_text_or_completion_object(response_v2)
text_v1 = wrapper_v2.extract_text_or_completion_object(response_v1)

print(f"\nExtracted text (V2): {text_v2}")
print(f"Extracted text (V1): {text_v1}")

## Summary

The Mistral V2 client provides:

✅ **Rich UnifiedResponse** with typed content blocks
✅ **Direct property access** (`response.text`, `response.cost`)
✅ **Tool call preservation** as `ToolCallContent` blocks
✅ **Backward compatibility** via `create_v1_compatible()` and `message_retrieval()`
✅ **Seamless agent integration** - works with all AG2 agents
✅ **Cost and usage tracking** built-in

The V2 client maintains full compatibility with existing AG2 code while providing access to rich content features.